In [1]:
from dabry.problem import NavigationProblem
from dabry.wind import DiscreteWind
from dabry.model import ZermeloGeneralModel
from dabry.misc import Utils
from dabry.ddf_manager import DDFmanager
from dabry.solver_ef import SolverEF
import numpy as np

In [6]:
i_member = 0
for i_member in range(51):
    wind = DiscreteWind()
    wind.load(f'../output/wind_scenarii2_{i_member}/wind.h5')
    model = ZermeloGeneralModel(23, coords='gcs')
    model.update_wind(wind)
    
    x_init = Utils.DEG_TO_RAD * np.array((-17.5, 14.7))
    x_target = Utils.DEG_TO_RAD * np.array((-35, -5.8))

    bl = Utils.DEG_TO_RAD * np.array((-40, -10))
    tr = Utils.DEG_TO_RAD * np.array((-10, 20))

    pb = NavigationProblem(model, x_init, x_target, 'gcs', bl=bl, tr=tr)

    ddf = DDFmanager(cache_wind=True)
    ddf.setup()
    ddf.set_case(f'wind_scenarii2_{i_member}')
    ddf.clean_output_dir()
    
    solver = SolverEF(pb, max_time=20*3600, max_steps=500)
    res = solver.solve()
    
    ddf.dump_trajs([res.traj])
    ddf.log(pb)

Problem | TIMEOPT | 23.00 m/s | 2.99e+06 m | scale 36h6m | orthodromic 41h1m
Steps :    168/500, Extremals :    233, Active :  152, Dist :  13 Comp : 1 Stopped empty active list
Target reached in 39h2m | +8% no wind | -4% orthodromic | cpu time 5.92s
Problem | TIMEOPT | 23.00 m/s | 2.99e+06 m | scale 36h6m | orthodromic 40h45m
Steps :    167/500, Extremals :    239, Active :  149, Dist :  13 Comp : 1 Stopped empty active list
Target reached in 39h2m | +8% no wind | -4% orthodromic | cpu time 5.68s
Problem | TIMEOPT | 23.00 m/s | 2.99e+06 m | scale 36h6m | orthodromic 38h9m
Steps :    160/500, Extremals :    241, Active :  163, Dist :  13 Comp : 1 Stopped empty active list
Target reached in 37h21m | +3% no wind | -2% orthodromic | cpu time 5.70s
Problem | TIMEOPT | 23.00 m/s | 2.99e+06 m | scale 36h6m | orthodromic 39h30m
Steps :    164/500, Extremals :    229, Active :  151, Dist :  13 Comp : 1 Stopped empty active list
Target reached in 38h12m | +5% no wind | -3% orthodromic | cpu tim

Steps :    174/500, Extremals :    241, Active :  150, Dist :  13 Comp : 1 Stopped empty active list
Target reached in 40h44m | +12% no wind | -3% orthodromic | cpu time 6.22s
Problem | TIMEOPT | 23.00 m/s | 2.99e+06 m | scale 36h6m | orthodromic 39h30m
Steps :    166/500, Extremals :    235, Active :  154, Dist :  13 Comp : 1 Stopped empty active list
Target reached in 38h28m | +6% no wind | -2% orthodromic | cpu time 5.85s
Problem | TIMEOPT | 23.00 m/s | 2.99e+06 m | scale 36h6m | orthodromic 41h22m
Steps :    168/500, Extremals :    241, Active :  157, Dist :  13 Comp : 1 Stopped empty active list
Target reached in 39h19m | +8% no wind | -4% orthodromic | cpu time 5.92s
Problem | TIMEOPT | 23.00 m/s | 2.99e+06 m | scale 36h6m | orthodromic 40h23m
Steps :    164/500, Extremals :    238, Active :  153, Dist :  13 Comp : 1 Stopped empty active list
Target reached in 38h12m | +5% no wind | -5% orthodromic | cpu time 5.92s
Problem | TIMEOPT | 23.00 m/s | 2.99e+06 m | scale 36h6m | orthod

In [7]:
import shutil
for i_member in range(51):
    try:
        shutil.copy(f'../output/wind_scenarii2_{i_member}/trajectories.h5',
                   f'../output/wind_scenarii2_all/trajectories_{i_member}.h5')
    except FileNotFoundError:
        pass
shutil.copy('../output/wind_scenarii2_0/params.json', '../output/wind_scenarii2_all/params.json')
shutil.copy('../output/wind_scenarii2_0/wind.h5', '../output/wind_scenarii2_all/wind.h5')

'../output/wind_scenarii2_all/wind.h5'

In [15]:
wind = DiscreteWind()
wind.load(f'../output/wind_scenarii_all/wind.h5')
model = ZermeloGeneralModel(23, coords='gcs')
model.update_wind(wind)

x_init = Utils.DEG_TO_RAD * np.array((-55, 40))
x_target = Utils.DEG_TO_RAD * np.array((-10, 40))

bl = Utils.DEG_TO_RAD * np.array((-60, 33))
tr = Utils.DEG_TO_RAD * np.array((-5, 55))

pb = NavigationProblem(model, x_init, x_target, 'gcs', bl=bl, tr=tr)

ddf = DDFmanager(cache_wind=True)
ddf.setup()
ddf.set_case(f'wind_scenarii_all')

from dabry.feedback import HTargetFB
from dabry.stoppingcond import DistanceSC
pb.load_feedback(HTargetFB(x_target, 'gcs'))
sc = DistanceSC(lambda x: (x - x_target) @ (x - x_target), (x_init - x_target) @ (x_init - x_target) / 500)
pb.integrate_trajectory(x_init, sc, t_init=pb.model.wind.t_start, int_step=3600/100)
ddf.dump_trajs(pb.trajs, filename='trajectories_heuristic.h5')
ddf.log(pb)